# Generating Initial Approximation for `a` in RWC_alam()

*Last updated by Arthur Ryman on 2022-05-19*

## Introduction

The function `RWC_alam(B, c1, c2, v)` computes optimal values for the radial wave function parameters
$(a, \lambda_v)$.
It does so by minimizing the expectation value $E(a,\lambda_v)$ of 
the RWC Hamiltonian defined by $(B, c1, c2)$ on the ground state.
The minimum occurs where the derivative of $E$ vanishes.
I originally implemented the Python version using the SymPy function `solveset()` since this should return all
zeros.

Unfortunately, `solveset()` failed to find solutions in some cases.
I therefore replaced it with `nsolve()` which uses numerical methods.
However, `nsolve()` does require a good initial approximation, otherwise it also fails to find a solution.
Also, it only returns the first solution it finds for the given initial approximation.
It is therefore important to compute a good initial approximation.

The goal of this notebook is to develop the theory of how to generate a good initial approximation.

All equation references in this notebook refer to WR2015.

In [1]:
from sympy import *
from acmpy.papers.wr2015 import *

## RWC Hamiltonians

We consider $SO(5)$-invariant Hamiltonians defined by (B.12) where $\chi = \kappa = 0$.

$$
\begin{gather}
H(B, c_1, c_2) = -\frac{1}{2B}\nabla^2 + \frac{B}{2}(c_1 \beta^2 + c_2 \beta^4)
\end{gather}
$$

Here the parameters $B, c_1, c_2$ are real numbers constrained as follows.

$$
\begin{gather}
B > 0 \\
c_2 \ge 0 \\
\mbox{if $c_2 = 0$ then $c_1 > 0$}
\end{gather}
$$

The expectation value of the Hamiltonian in the ground state is given by (B.16).

In [2]:
Eq_B16

Eq(E, B*c1*lambda0/(2*a**2) + B*c2*lambda0*(lambda0 + 1)/(2*a**4) + a**2*(1 + 9/(4*lambda0 - 4))/(2*B))

Our goal is to compute the value $a_0$ of $a$ that minimizes $E(a, \lambda_0(a),B,c_1,c_2)$.

$$
a_0 = \arg\min_a E(a, \lambda_0(a),B,c_1,c_2)
$$

In [3]:
a0 = Symbol('a0', real=True, positive=True)
a0

a0

Observe that $E$ depends on $a^2$. 
Introduce the variable $A = a^2$.

In [4]:
def A_a(a):
    return a ** 2
    
A = Symbol('A', real=True, positive=True)
Eq_A = Eq(A, A_a(a))
Eq_A

Eq(A, a**2)

In [5]:
def a_A(A):
    return sqrt(A)
Eq_a = Eq(a, a_A(A))
Eq_a

Eq(a, sqrt(A))

Define $F$ such that $F(A) = F(a^2) = E(a)$.

In [6]:
F = Symbol('F', real=True)
F

F

In [7]:
def F_B16(A, lambda0, B, c1, c2):
    F = E_B16(sqrt(A), lambda0, B, c1, c2)

    return F

Eq(F, F_B16(A, lambda0, B, c1, c2))

Eq(F, A*(1 + 9/(4*lambda0 - 4))/(2*B) + B*c1*lambda0/(2*A) + B*c2*lambda0*(lambda0 + 1)/(2*A**2))

In [8]:
F_B16(a ** 2, lambda0, B, c1, c2) == E_B16(a, lambda0, B, c1, c2)

True

Let $A_0$ be the value of $A$ that minimizes $F(A, \lambda_0(\sqrt{A}), B, c_1, c_2)$.

$$
A_0 = \arg\min_A F(A, \lambda_0(\sqrt{A}),B,c_1,c_2)
$$

In [9]:
A0 = Symbol('A0', real=True, positive=True)
A0

A0

Since $A$ is a monotonic, strictly increasing function of $a$,
we have

$$
A_0 = a_0^2
$$

The parameter $\lambda_0$ is defined by (B.11).

In [10]:
Eq_B11

Eq(lambda0, sqrt(a**4*beta0**4 + 9/4) + 1)

The parameter $\beta_0$ is defined by (B.15).

In [11]:
Eq_B15

Eq(beta0, Piecewise((sqrt(2)*sqrt(-c1/c2)/2, c1 < 0), (0, True)))

Finally, $\lambda_v$ is computed from $\lambda_0$ and the seniority $v$ using one of several possible
rules.

The main implementation uses the rule defined by (61).

In [12]:
Eq_61

Eq(lambda_v, lambda0 + v)

The even-odd parity rule is defined by (62).

In [13]:
Eq_62

Eq(lambda_v, lambda0 + Mod(v, 2))

The constant rule is defined by (63).

In [14]:
Eq_63

Eq(lambda_v, lambda0)

## Generating an Initial Approximation


The approach we use is to approximate the expectation value $E(a, \lambda_0, B, c_1, c_2)$ 
by a function that agrees with it for very small and very large values of $a$, 
and to use its minimum as the initial guess for the minimum of the actual expectation value.

There are three cases of the parameters to consider:
 1. $c_2 = 0, c_1 > 0$
 2. $c_2 > 0, c_1 >= 0$
 3. $c_2 > 0, c_1 < 0$

## Case 1: $c_2 = 0, c_1 > 0$

Define the parameters for this case.

In [15]:
c1_case1 = Symbol('c1', real=True, positive=True)
c2_case1 = 0

Use (B.15) to compute $\beta_0$ for this case.

In [16]:
beta0_case1 = beta0_B15(c1_case1, c2_case1)
Eq(beta0, beta0_case1)

Eq(beta0, 0)

Use (B.11) to compute $\lambda_0$ for this case.

In [17]:
lambda0_case1 = lambda0_B11(sqrt(A), beta0_case1)
Eq(lambda0, lambda0_case1)

Eq(lambda0, 5/2)

Use (B.16) to compute $F$ for this case.

In [18]:
F_case1 = F_B16(A, lambda0_case1, B, c1_case1, c2_case1)
Eq(F, F_case1)

Eq(F, 5*A/(4*B) + 5*B*c1/(4*A))

Compute the derivative of $F$ and find its zeros.

In [19]:
DF_case1 = F_case1.diff(A)
DF_case1

5/(4*B) - 5*B*c1/(4*A**2)

We can find the zeros exactly in this case. There is no need to call `nsolve()`.

In [20]:
solutions_case1 = solveset(DF_case1, A, domain=Reals)
solutions_case1

{-B*sqrt(c1), B*sqrt(c1)}

Observe that `solveset()` does not use the assumptions on $A$ and as a consequence returns
a negative solution which we must eliminate.

In [21]:
feasible_solutions_case1 = [s for s in solutions_case1 if s.is_positive]
feasible_solutions_case1

[B*sqrt(c1)]

In [22]:
Eq_case1 = Eq(A0, feasible_solutions_case1[0])
Eq_case1

Eq(A0, B*sqrt(c1))

As a check, solve the equation using `solve()` which does respect the assumptions on $A$.

In [23]:
solve(DF_case1, A)

[B*sqrt(c1)]

## Case 2: $c_2 > 0, c_1 >= 0$

Introduce the variables for this case.

In [24]:
c1_case2 = Symbol('c1', real=True, nonnegative=True)
c2_case2 = Symbol('c2', real=True, positive=True)

In [25]:
beta0_case2 = beta0_B15(c1_case2, c2_case2)
beta0_case2

0

In [26]:
lambda0_case2 = lambda0_B11(sqrt(A), beta0_case2)
lambda0_case2

5/2

In [27]:
F_case2 = F_B16(A, lambda0_case2, B, c1_case2, c2_case2)
F_case2

5*A/(4*B) + 5*B*c1/(4*A) + 35*B*c2/(8*A**2)

In [28]:
DF_case2 = F_case2.diff(A)
DF_case2

5/(4*B) - 5*B*c1/(4*A**2) - 35*B*c2/(4*A**3)

The zeros of this expression can be found by solving a cubic equation.
However, the general solution of a cubic polynomial is very complicated and involves complex numbers.
Compute the solution uses `solveset()` and `solve()`.

In [29]:
solveset_case2 = solveset(DF_case2, A, domain=Reals)
solveset_case2

Complement(Intersection({B**2*c1/(3*(7*B**2*c2/2 + sqrt(-B**6*c1**3/27 + 49*B**4*c2**2/4))**(1/3)) + (7*B**2*c2/2 + sqrt(-B**6*c1**3/27 + 49*B**4*c2**2/4))**(1/3), B**2*c1/(3*(-1/2 - sqrt(3)*I/2)*(7*B**2*c2/2 + sqrt(-B**6*c1**3/27 + 49*B**4*c2**2/4))**(1/3)) + (-1/2 - sqrt(3)*I/2)*(7*B**2*c2/2 + sqrt(-B**6*c1**3/27 + 49*B**4*c2**2/4))**(1/3), B**2*c1/(3*(-1/2 + sqrt(3)*I/2)*(7*B**2*c2/2 + sqrt(-B**6*c1**3/27 + 49*B**4*c2**2/4))**(1/3)) + (-1/2 + sqrt(3)*I/2)*(7*B**2*c2/2 + sqrt(-B**6*c1**3/27 + 49*B**4*c2**2/4))**(1/3)}, Reals), {0})

In [30]:
solve_case2 = solve(DF_case2, A)
solve_case2

[B**2*c1/(3*(7*B**2*c2/2 + sqrt(-B**6*c1**3/27 + 49*B**4*c2**2/4))**(1/3)) + (7*B**2*c2/2 + sqrt(-B**6*c1**3/27 + 49*B**4*c2**2/4))**(1/3),
 B**2*c1/(3*(-1/2 - sqrt(3)*I/2)*(7*B**2*c2/2 + sqrt(-B**6*c1**3/27 + 49*B**4*c2**2/4))**(1/3)) + (-1/2 - sqrt(3)*I/2)*(7*B**2*c2/2 + sqrt(-B**6*c1**3/27 + 49*B**4*c2**2/4))**(1/3),
 B**2*c1/(3*(-1/2 + sqrt(3)*I/2)*(7*B**2*c2/2 + sqrt(-B**6*c1**3/27 + 49*B**4*c2**2/4))**(1/3)) + (-1/2 + sqrt(3)*I/2)*(7*B**2*c2/2 + sqrt(-B**6*c1**3/27 + 49*B**4*c2**2/4))**(1/3)]

In [31]:
solve_case2[0]

B**2*c1/(3*(7*B**2*c2/2 + sqrt(-B**6*c1**3/27 + 49*B**4*c2**2/4))**(1/3)) + (7*B**2*c2/2 + sqrt(-B**6*c1**3/27 + 49*B**4*c2**2/4))**(1/3)

Rather that use the exact solution, we use `nsolve()`.
Compute the approximations to $DF$ for small and large $A$.

For small $A$, $DF(A)$ is of order $A^{-3}$.

In [32]:
O(DF_case2, (A, 0))

O(A**(-3))

In [33]:
DF_case2_approx_0 = limit(DF_case2 * A ** 3, A, 0) / A ** 3
DF_case2_approx_0

-35*B*c2/(4*A**3)

which is a large negative value.

For large $A$ $DF(A)$ is constant.

In [34]:
O(DF_case2, (A, oo))

O(1, (A, oo))

In [35]:
DF_case2_approx_oo = limit(DF_case2, A, oo)
DF_case2_approx_oo

5/(4*B)

which is a positive value.

Since $DF(A)$ is negative for small $A$ and positive for large $A$, it must have at least one zero.
Approximate $DF$ as the sum these functions.

In [36]:
DF_case2_approx = DF_case2_approx_0 + DF_case2_approx_oo
DF_case2_approx

5/(4*B) - 35*B*c2/(4*A**3)

Find the zeros of this approximation.
First use `solveset()`.

In [37]:
solveset_case2_approx = solveset(DF_case2_approx, A, domain=Reals)
solveset_case2_approx

{7**(1/3)*B**(2/3)*c2**(1/3)}

In [38]:
A0_solveset_case2_approx = solveset_case2_approx.args[0]
A0_solveset_case2_approx

7**(1/3)*B**(2/3)*c2**(1/3)

which is a positive value.

Therefore $DF$ has at least one positive zero.

Verify this solution using `solve()`.

In [39]:
solve_case2_approx = solve(DF_case2_approx, A)
solve_case2_approx

[7**(1/3)*B**(2/3)*c2**(1/3)]

In [40]:
A0_solve_case2_approx = solve_case2_approx[0]
A0_solve_case2_approx

7**(1/3)*B**(2/3)*c2**(1/3)

## Case 3: $c_2 > 0, c_1 < 0$

Introduce the variables for this case.

In [41]:
c1_case3 = Symbol('c1', real=True, negative=True)
c2_case3 = Symbol('c2', real=True, positive=True)

beta0_case3 = beta0_B15(c1_case3, c2_case3)
beta0_case3

sqrt(2)*sqrt(-c1)/(2*sqrt(c2))

In [42]:
lambda0_case3 = lambda0_B11(sqrt(A), beta0_case3)
lambda0_case3

sqrt(A**2*c1**2/(4*c2**2) + 9/4) + 1

In [43]:
F_case3 = F_B16(A, lambda0_case3, B, c1_case3, c2_case3)
F_case3

A*(1 + 9/(4*sqrt(A**2*c1**2/(4*c2**2) + 9/4)))/(2*B) + B*c1*(sqrt(A**2*c1**2/(4*c2**2) + 9/4) + 1)/(2*A) + B*c2*(sqrt(A**2*c1**2/(4*c2**2) + 9/4) + 1)*(sqrt(A**2*c1**2/(4*c2**2) + 9/4) + 2)/(2*A**2)

In [44]:
DF_case3 = F_case3.diff(A)
DF_case3

-9*A**2*c1**2/(32*B*c2**2*(A**2*c1**2/(4*c2**2) + 9/4)**(3/2)) + B*c1**3/(8*c2**2*sqrt(A**2*c1**2/(4*c2**2) + 9/4)) + (1 + 9/(4*sqrt(A**2*c1**2/(4*c2**2) + 9/4)))/(2*B) + B*c1**2*(sqrt(A**2*c1**2/(4*c2**2) + 9/4) + 1)/(8*A*c2*sqrt(A**2*c1**2/(4*c2**2) + 9/4)) + B*c1**2*(sqrt(A**2*c1**2/(4*c2**2) + 9/4) + 2)/(8*A*c2*sqrt(A**2*c1**2/(4*c2**2) + 9/4)) - B*c1*(sqrt(A**2*c1**2/(4*c2**2) + 9/4) + 1)/(2*A**2) - B*c2*(sqrt(A**2*c1**2/(4*c2**2) + 9/4) + 1)*(sqrt(A**2*c1**2/(4*c2**2) + 9/4) + 2)/A**3

For small $A$, $DF(A)$ has order $A^{-3}$.

In [45]:
O(DF_case3, (A, 0))

O(A**(-3))

DF_case3_approx_0 = limit(DF_case3 * A ** 3, A, 0) / A ** 3
DF_case3_approx_0

For large $A$, $DF(A)$ is constant.

In [46]:
O(DF_case3, (A, oo))

O(1, (A, oo))

In [47]:
DF_case3_approx_oo = limit(DF_case3, A, oo)
DF_case3_approx_oo

1/(2*B)

Approximate $DF$ as the sum of these two limits.

DF_case3_approx = DF_case3_approx_0 + DF_case3_approx_oo
DF_case3_approx

Find the zeros of the approximation to $DF$ using `solveset()` and `solve()`.

In [48]:
solveset_case3_approx = solveset(DF_case3_approx, A, domain=Reals)
solveset_case3_approx

NameError: name 'DF_case3_approx' is not defined

In [ ]:
A0_solveset_case3_approx = solveset_case3_approx.args[0]
A0_solveset_case3_approx

In [ ]:
solve_case3_approx = solve(DF_case3_approx, A)
solve_case3_approx

In [ ]:
A0_solve_case3_approx = solve_case3_approx[0]
A0_solve_case3_approx